## Importing packages

In [27]:
import numpy as np
import pandas as pd
import twitter
from twitter import Twitter
from twitter import OAuth

import json 
from pandas.io.json import json_normalize

ck = 'QvGBrXMEZhicazT2XzvK1usgY' #consumer key
cs = 'TyYiFfvYszrjtoikhn5dlMYWljuEPGQu3iUlPZmByZAgKwTgP2' #consumer key secret
at = '954075809018470400-2TRhibAbIA7OBhPUXuf6xE8NIMTWa17' 
ats = 'qbg2ibQ4QILAIYidy9kdANZ5cxSO2a9jCPhOIYmsGm1Wv'

oauth = OAuth(at,ats,ck,cs)

api = Twitter(auth=oauth)

## Extracting the tweets using API

In [28]:
df = pd.DataFrame()
mid = 0
for i in range(10):
    if i ==0:
        search_tw = api.search.tweets(q="Bitcoin", count = 100)
    else:
        search_tw = api.search.tweets(q="Bitcoin", count=100, max_id=mid)
    
    dftemp = json_normalize(search_tw,'statuses')
    mid = dftemp['id'].min()
    mid=mid-1
    df = df.append(dftemp,ignore_index=True)

In [29]:
df.shape

(912, 30)

## Normalizing the fetched JSON data

In [30]:
tweet = df['text']
df_u = json_normalize(df['user'])
df_u.head()
df_s = df_u['screen_name']
df_s.head()
df['screenname']=df_u['screen_name']

## Importing NLP tool kit package Textblob

In [33]:
from textblob import TextBlob as tb

## Determining sentiment of tweets

In [34]:
pol = []
sub = []

for j in tweet:
    tx = tb(j)
    pol.append(tx.sentiment.polarity)
    sub.append(tx.sentiment.subjectivity)

In [35]:
df_pols = pd.DataFrame({"polarity":pol,"subjectivity":sub})
df['polarity']=df_pols['polarity']
df['subjectivity']=df_pols['subjectivity']

In [36]:
df_sup = pd.DataFrame()
df3 = pd.DataFrame()

In [37]:
df3 = df[['polarity','subjectivity','screenname']]
df3.head()

,polarity,subjectivity,screenname
0,0.0,0.0,coindromedabtc
1,0.0,0.0,Bitmex0805
2,0.0,0.0,dden_official
3,0.5,0.5,PlsGiveBitcoin
4,0.0,0.0,BitcoinEdu


## Classifying tweets into positive, negative, and neutral

In [39]:
negative = pd.DataFrame()
positive= pd.DataFrame()
neutral = pd.DataFrame()

negative = df3[df3['polarity']<=-0.5]
positive = df3[df3['polarity']>=0.5]
neutral = df3[(df3['polarity'] > -0.5) & (df3['polarity'] < 0.5)]

In [40]:
negative.head()

,polarity,subjectivity,screenname
43,-0.80,0.9,AltcoinTicker
238,-0.75,1.0,sukotonkotsu
239,-0.80,1.0,alzex6181
253,-0.75,1.0,sky_fly_ripple
284,-0.50,1.0,rBitcoins


In [41]:
positive.head()

,polarity,subjectivity,screenname
3,0.500000,0.5,PlsGiveBitcoin
48,0.520833,1.0,kola3wd
53,0.500000,0.5,cryptonizuka
57,0.500000,0.4,mariell21547124
64,0.500000,0.5,KLRTRDR


In [42]:
neutral.head()

,polarity,subjectivity,screenname
0,0.0,0.0,coindromedabtc
1,0.0,0.0,Bitmex0805
2,0.0,0.0,dden_official
4,0.0,0.0,BitcoinEdu
5,0.0,0.0,btcreports
